In [42]:
from base_models.modeling_gpt_neox import GPTNeoXForCausalLM
from flash_attn.models.gpt import GPTLMHeadModel

from flash_attn.models.gpt_neox import gpt_neox_config_to_gpt2_config, remap_state_dict_hf_gpt_neox

In [15]:
import sys
sys.path.append('..')

In [16]:
model_name = "EleutherAI/pythia-70m-deduped"
model = GPTNeoXForCausalLM.from_pretrained(model_name)

In [34]:
# NotImplementedError
# flash_model = GPTLMHeadModel.from_pretrained(config=config, model_name=model_name)

In [35]:
config = gpt_neox_config_to_gpt2_config(model.config)
flash_model = GPTLMHeadModel(config=config)

In [36]:
# # keys mismatch
# flash_model.load_state_dict(model.state_dict())

In [40]:
state_dict = model.state_dict()

In [50]:
new_state_dict = remap_state_dict_hf_gpt_neox(state_dict, config=config)

KeyError: 'transformer.layers.0.attention.bias'

In [48]:
# for n, p in flash_model.named_parameters():
#     print(n, p.shape)

In [ ]:
for n, p in model.named_parameters():
    print(n, p.shape)

gpt_neox.embed_in.weight torch.Size([50304, 512])
gpt_neox.layers.0.input_layernorm.weight torch.Size([512])
gpt_neox.layers.0.input_layernorm.bias torch.Size([512])
gpt_neox.layers.0.post_attention_layernorm.weight torch.Size([512])
gpt_neox.layers.0.post_attention_layernorm.bias torch.Size([512])
gpt_neox.layers.0.attention.query_key_value.weight torch.Size([1536, 512])
gpt_neox.layers.0.attention.query_key_value.bias torch.Size([1536])
gpt_neox.layers.0.attention.dense.weight torch.Size([512, 512])
gpt_neox.layers.0.attention.dense.bias torch.Size([512])
gpt_neox.layers.0.mlp.dense_h_to_4h.weight torch.Size([2048, 512])
gpt_neox.layers.0.mlp.dense_h_to_4h.bias torch.Size([2048])
gpt_neox.layers.0.mlp.dense_4h_to_h.weight torch.Size([512, 2048])
gpt_neox.layers.0.mlp.dense_4h_to_h.bias torch.Size([512])
gpt_neox.layers.1.input_layernorm.weight torch.Size([512])
gpt_neox.layers.1.input_layernorm.bias torch.Size([512])
gpt_neox.layers.1.post_attention_layernorm.weight torch.Size([512])

In [1]:
from transformers import AutoModelForCausalLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

/home/jovyan/rmt/env_cuda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "gpt2"
tokenizer_name_or_path = "gpt2"

In [3]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    inference_mode=False, 
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1
)

In [4]:

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
# output: trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282

/home/jovyan/rmt/env_cuda/lib/python3.8/site-packages/peft/tuners/lora.py:299: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.23643136409814364


In [ ]:
peft_lora = get_peft_model(base_model, peft_config_lora)
peft_prompt_tuning = get_peft_model(peft_lora, peft_config_prompt_tuning)